# 02_entiry-extract

##install extra library

In [ ]:
!pip install transformers torch
!pip install sentencepiece

##read stored data from DataPlatform

In [ ]:
from dsmlibrary.datanode import DataNode
import json
from tqdm.auto import tqdm
import pandas as pd
import re

In [ ]:
dir_raw_id = 62
dir_process_id = 63

In [ ]:
datanode = DataNode()

In [ ]:
meta, fp = datanode.get_file(datanode.get_file_id(directory_id=dir_raw_id, name='eventseye.json'))
meta

In [ ]:
data = json.load(fp)
data

##NER (NameEntitiesRegcognition)

In [ ]:
from transformers import AutoTokenizer, AutoModelForTokenClassification
from transformers import pipeline

In [ ]:
tokenizer = AutoTokenizer.from_pretrained("Davlan/xlm-roberta-base-ner-hrl")
model = AutoModelForTokenClassification.from_pretrained("Davlan/xlm-roberta-base-ner-hrl")
nlp = pipeline("ner", model=model, tokenizer=tokenizer)

In [ ]:
example = "LUXURY BRIDAL EXPO GEORGIOS BANQUETS ORLAND PARK"

ner_results = nlp(example)
print(ner_results)

In [ ]:
def post_process(preds):
    data = []
    for i in range(len(preds)): 
        a = preds[i]
        text = a.get('word')
        for j in range(i, len(preds)):
            b = preds[j]
            if b.get('start') == a.get('end'):
                text += b.get('word')
                a = b
        data.append({
            'type': a.get('entity'),
            'text': text
        })
    tmp = []
    for elm in data:
        text = elm.get('text')
        if bool(re.match("[ก-๛]|▁", text)):
            elm.update({'text': text.replace('▁', ' ')})
            tmp.append(elm)
    return tmp

In [ ]:
text = data[5]['Country']
text

In [ ]:
output = nlp(text)
output

In [ ]:
post_process(output)

##Process eventseye to spo

In [ ]:
spo = []
for eventseye in tqdm(data):
    Exhibition_Name = eventseye.get('Exhibition_Name') 
    if Exhibition_Name == None:
        continue
    ner = nlp(Exhibition_Name)
    processed = post_process(ner)
    for kw in processed:
        spo.append({
            'Exhibition_Name': eventseye.get('Exhibition_Name', ""),
            'Describe': eventseye.get('Describe', ""),
            'Country': eventseye.get('Country', ""),
            'City': eventseye.get('City', ""),
            'Place': eventseye.get('Place', ""),
        })

In [ ]:
df = pd.DataFrame(spo)
df['Exhibition_Name'] = df['Exhibition_Name'].apply(lambda x: x[2:])
df

In [ ]:
#write DataNode to DataPlatform

In [ ]:
datanode.write(df=df, directory=dir_process_id, name="spo", profiling=True, lineage=[datanode.get_file_id(directory_id=dir_raw_id, 
                                                                                                          name='eventseye.json')])